# Zonal Averaging

This section demonstrates how to perform Zonal Averaging using UXarray, covering both non-conservative and conservative methods.


In [1]:
import matplotlib.pyplot as plt
import numpy as np

import uxarray as ux

uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

## What is a Zonal Average/Mean?

A zonal average (or zonal mean) is a statistical measure that represents the average of a variable along lines of constant latitude or over latitudinal bands.

UXarray provides two types of zonal averaging:
- **Non-conservative**: Samples values at specific latitude lines
- **Conservative**: Preserves integral quantities by weighting faces by their area overlap with latitude bands

```{seealso}
[NCL Zonal Average](https://www.ncl.ucar.edu/Applications/zonal.shtml)
```

## Non-Conservative Zonal Averaging

The non-conservative method samples values at specific lines of constant latitude. This is the default behavior and is suitable for visualization and general analysis where exact conservation is not required.

Let's first visualize our data field and then demonstrate zonal averaging:

In [2]:
# Display the global field
uxds["psi"].plot(cmap="inferno", periodic_elements="split", title="Global Field")

# Show conceptual latitude bands for zonal averaging
print("Latitude bands for zonal averaging:")
lat_bands = np.arange(-90, 91, 30)
for i, lat in enumerate(lat_bands[:-1]):
    print(f"Band {i + 1}: {lat}° to {lat_bands[i + 1]}°")

Latitude bands for zonal averaging:
Band 1: -90° to -60°
Band 2: -60° to -30°
Band 3: -30° to 0°
Band 4: 0° to 30°
Band 5: 30° to 60°
Band 6: 60° to 90°


In [3]:
zonal_mean_psi = uxds["psi"].zonal_mean()
zonal_mean_psi

/tmp/ipykernel_4141335/491649382.py:1: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  zonal_mean_psi = uxds["psi"].zonal_mean()


<xarray.DataArray 'psi_zonal_mean' (latitudes: 19)> Size: 152B
array([1.08546247, 1.09219987, 1.10863654, 1.10011333, 1.04989243,
       1.03074397, 1.01980815, 1.01312147, 0.9973705 , 1.00000001,
       1.00262952, 0.98687854, 0.98019186, 0.96925608, 0.95010754,
       0.89988665, 0.89136345, 0.90780013, 0.91453753])
Coordinates:
  * latitudes  (latitudes) int64 152B -90 -80 -70 -60 -50 -40 ... 50 60 70 80 90
Attributes:
    zonal_mean:    True
    conservative:  False

The default latitude range is between -90 and 90 degrees with a step size of 10 degrees. 

In [4]:
lats = zonal_mean_psi["latitudes"].values
vals = zonal_mean_psi.values
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(lats, vals, "-")
ax.scatter(lats, vals, s=12, color="red")
ax.set_title("Zonal Average Plot (Default)")
ax.set_xlim(-95, 95)
ax.set_xticks(np.arange(-90, 100, 20))

The range of latitudes can be modified by using the `lat` parameter. It accepts:

* **Single scalar**: e.g., `lat=45`
* **List/array**: e.g., `lat=[10, 20]` or `lat=np.array([10, 20])`
* **Tuple**: e.g., `(min_lat, max_lat, step)`

In [5]:
zonal_mean_psi_large = uxds["psi"].zonal_mean(lat=(-90, 90, 1))

/tmp/ipykernel_4141335/322622288.py:1: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  zonal_mean_psi_large = uxds["psi"].zonal_mean(lat=(-90, 90, 1))


In [6]:
lats = zonal_mean_psi_large["latitudes"].values
vals = zonal_mean_psi_large.values
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(lats, vals, "-")
ax.scatter(lats, vals, s=4, color="red")
ax.set_title("Zonal Average Plot (Larger Sample)")
ax.set_xlim(-95, 95)
ax.set_xticks(np.arange(-90, 100, 20))

## Conservative Zonal Averaging

Conservative zonal averaging preserves integral quantities (mass, energy, momentum) by computing area-weighted averages over latitude bands. This is essential for climate model analysis, energy budget calculations, and any application requiring physical conservation.

### Key Differences from Non-Conservative:
- **Non-conservative**: Samples at specific latitude lines
- **Conservative**: Averages over latitude bands between adjacent lines
- **Conservation**: Preserves global integrals to machine precision

In [7]:
# Conservative zonal averaging with bands
bands = np.array([-90, -60, -30, 0, 30, 60, 90])
conservative_result = uxds["psi"].zonal_mean(lat=bands, conservative=True)
conservative_result

/tmp/ipykernel_4141335/2677525412.py:3: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  conservative_result = uxds["psi"].zonal_mean(lat=bands, conservative=True)


<xarray.DataArray 'psi_zonal_mean' (latitudes: 6)> Size: 48B
array([1.10364701, 1.03903852, 1.00581915, 0.9937623 , 0.96140404,
       0.89634629])
Coordinates:
  * latitudes  (latitudes) float64 48B -75.0 -45.0 -15.0 15.0 45.0 75.0
Attributes:
    zonal_mean:      True
    conservative:    True
    lat_band_edges:  [-90. -60. -30.   0.  30.  60.  90.]

### Conservation Verification

A key advantage of conservative zonal averaging is that it preserves global integrals.

In [8]:
# Test conservation property
global_mean = uxds["psi"].mean()
full_sphere_conservative = uxds["psi"].zonal_mean(lat=[-90, 90], conservative=True)
conservation_error = abs(global_mean.values - full_sphere_conservative.values[0])

print(f"Global mean: {global_mean.values:.12f}")
print(f"Conservative full sphere: {full_sphere_conservative.values[0]:.12f}")
print(f"Conservation error: {conservation_error:.2e}")

Global mean: 1.000000001533
Conservative full sphere: 1.000000001829
Conservation error: 2.96e-10


/tmp/ipykernel_4141335/1814533177.py:3: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  full_sphere_conservative = uxds["psi"].zonal_mean(lat=[-90, 90], conservative=True)


### Understanding the lat Parameter

Both conservative and non-conservative modes can use the same `lat` parameter, but they interpret it differently:

- **Non-conservative**: Creates sample points at the specified latitudes
- **Conservative**: Uses the latitudes as band edges, creating bands between adjacent points

In [9]:
# Demonstrate signature behavior
lat_tuple = (-90, 90, 30)  # Every 30 degrees

# Non-conservative: samples at lines
non_cons_lines = uxds["psi"].zonal_mean(lat=lat_tuple)
print(f"Non-conservative with lat={lat_tuple}:")
print(f"Sample points: {non_cons_lines.coords['latitudes'].values}")
print(f"Count: {len(non_cons_lines.coords['latitudes'].values)} points")

# Conservative: creates bands between lines
cons_bands = uxds["psi"].zonal_mean(lat=lat_tuple, conservative=True)
print(f"\nConservative with lat={lat_tuple}:")
print(f"Band centers: {cons_bands.coords['latitudes'].values}")
print(f"Count: {len(cons_bands.coords['latitudes'].values)} bands")

Non-conservative with lat=(-90, 90, 30):
Sample points: [-90 -60 -30   0  30  60  90]
Count: 7 points

Conservative with lat=(-90, 90, 30):
Band centers: [-75. -45. -15.  15.  45.  75.]
Count: 6 bands


/tmp/ipykernel_4141335/162242150.py:5: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  non_cons_lines = uxds["psi"].zonal_mean(lat=lat_tuple)
/tmp/ipykernel_4141335/162242150.py:11: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  cons_bands = uxds["psi"].zonal_mean(lat=lat_tuple, conservative=True)


### Visual Comparison: Conservative vs Non-Conservative

The differences between methods reflect their fundamental approaches:

- **Conservative**: More accurate for physical quantities because it accounts for the actual area of each face within latitude bands
- **Non-conservative**: Faster but approximates by sampling at specific latitude lines

The differences you see indicate how much area-weighting matters for your specific data and grid resolution.

In [10]:
# Compare with non-conservative at same latitudes
band_centers = 0.5 * (bands[:-1] + bands[1:])
non_conservative_comparison = uxds["psi"].zonal_mean(lat=band_centers)

# Create comparison plot
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(
    conservative_result.coords["latitudes"],
    conservative_result.values,
    "o-",
    label="Conservative (bands)",
    linewidth=3,
    markersize=8,
    color="red",
)
ax.plot(
    non_conservative_comparison.coords["latitudes"],
    non_conservative_comparison.values,
    "s--",
    label="Non-conservative (lines)",
    linewidth=2,
    markersize=7,
    color="blue",
    alpha=0.7,
)

ax.set_xlabel("Latitude (degrees)")
ax.set_ylabel("Zonal Mean Value")
ax.set_title("Conservative vs Non-Conservative Zonal Averaging")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xlim(-90, 90)

plt.tight_layout()
plt.show()

/tmp/ipykernel_4141335/523619492.py:3: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  non_conservative_comparison = uxds["psi"].zonal_mean(lat=band_centers)


### Understanding the Differences

The differences between conservative and non-conservative results depend on several factors:

1. **Grid Resolution**: Higher resolution grids show smaller differences
2. **Data Variability**: Rapidly changing fields show larger differences
3. **Latitude Band Size**: Wider bands increase the importance of area-weighting

**Which is more accurate?**
- **Conservative**: More accurate for physical quantities (mass, energy, momentum) because it preserves integral properties
- **Non-conservative**: Adequate for visualization and qualitative analysis

**When differences matter most:**
- Variable resolution grids (where face sizes vary significantly)
- Physical conservation requirements
- Quantitative analysis and budget calculations

In [11]:
# Quantify the differences
differences = conservative_result.values - non_conservative_comparison.values
max_diff = np.max(np.abs(differences))
mean_diff = np.mean(np.abs(differences))

print(f"Maximum absolute difference: {max_diff:.6f}")
print(f"Mean absolute difference: {mean_diff:.6f}")
print(
    f"Relative difference (max): {max_diff / np.mean(np.abs(conservative_result.values)) * 100:.3f}%"
)

Maximum absolute difference: 0.010503
Mean absolute difference: 0.006114
Relative difference (max): 1.050%


## Combined Plots

It is often desired to plot the zonal average along side other plots, such as color or contour plots. 

In [12]:
# Display the global field
print("Global Field:")
uxds["psi"].plot(cmap="inferno", periodic_elements="split", title="Global Field")

# Create zonal average plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(
    zonal_mean_psi.values,
    zonal_mean_psi.coords["latitudes"],
    "o-",
    linewidth=2,
    markersize=6,
    color="blue",
    label="Zonal Mean",
)

ax.set_xlabel("Zonal Mean Value")
ax.set_ylabel("Latitude (degrees)")
ax.set_title("Zonal Average")
ax.grid(True, alpha=0.3)
ax.set_ylim(-90, 90)
ax.legend()

# Add reference latitude lines
sample_bands = [-60, -30, 0, 30, 60]
for lat_band in sample_bands:
    ax.axhline(y=lat_band, color="red", linestyle="--", alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

Global Field:


## HEALPix Band Demonstration (20° band over USA latitudes)

We construct a low-resolution HEALPix grid, define a 20° latitude band spanning roughly the continental USA (32°N–52°N), compare conservative vs non-conservative zonal averaging at the band center (42°N), and visualize the band boundaries.

In [13]:
import cartopy.crs as ccrs

import uxarray as ux

uxgrid = ux.Grid.from_healpix(zoom=3, pixels_only=False)
data = np.sin(np.deg2rad(uxgrid.face_lat.values))
uxda = ux.UxDataArray(data, uxgrid=uxgrid, dims=["n_face"], name="val")

band_edges = np.array([32.0, 52.0])
lat_center = 42.0

z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
z_noncons = uxda.zonal_mean(lat=lat_center)
print("Conservative band mean [32N,52N]:", float(z_cons.values))
print("Non-conservative at 42N:", float(z_noncons.values))

fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
        label=f"lat={lat}°",
    )
lon = uxgrid.face_lon.values
lat = uxgrid.face_lat.values
mask = (lat >= 20) & (lat <= 60) & (lon >= -140) & (lon <= -60)
sc = ax.scatter(
    lon[mask],
    lat[mask],
    c=data[mask],
    s=8,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
ax.set_title("HEALPix face centers with 32–52°N band and center 42°N")
plt.colorbar(sc, ax=ax, orientation="horizontal", pad=0.05, label="val")

Conservative band mean [32N,52N]: 0.6527189003349049
Non-conservative at 42N: 0.6686793861174745


/tmp/ipykernel_4141335/1777304972.py:13: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
/tmp/ipykernel_4141335/1777304972.py:14: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_noncons = uxda.zonal_mean(lat=lat_center)
/tmp/ipykernel_4141335/1777304972.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Conservative band mean [32N,52N]:', float(z_cons.values))
/tmp/ipykernel_4141335/1777304972.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

In [14]:
# Highlight cut faces at band boundaries using face latitude bounds
fb = uxgrid.face_bounds_lat.values  # shape (n_face, 2) [min,max]
cut_lo = (fb[:, 0] < band_edges[0]) & (fb[:, 1] > band_edges[0])
cut_hi = (fb[:, 0] < band_edges[1]) & (fb[:, 1] > band_edges[1])
cut = cut_lo | cut_hi
faces = np.where(cut)[0]

# Draw polygons for a subset around CONUS
fnc = uxgrid.face_node_connectivity.values
node_lon = uxgrid.node_lon.values
node_lat = uxgrid.node_lat.values
fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
for f in faces:
    nodes = fnc[f]
    nodes = nodes[nodes >= 0]
    poly_lon = node_lon[nodes]
    poly_lat = node_lat[nodes]
    ax.plot(
        np.r_[poly_lon, poly_lon[0]],
        np.r_[poly_lat, poly_lat[0]],
        color="crimson",
        linewidth=0.8,
        transform=ccrs.PlateCarree(),
        alpha=0.7,
    )
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
    )
ax.set_title("Faces intersecting 32°N / 52°N band edges (cut region)")

Text(0.5, 1.0, 'Faces intersecting 32°N / 52°N band edges (cut region)')

## HEALPix Band Demonstration (20° band over USA latitudes)

We construct a low-resolution HEALPix grid, define a 20° latitude band spanning roughly the continental USA (32°N–52°N), compare conservative vs non-conservative zonal averaging at the band center (42°N), and visualize the band boundaries.

In [15]:
import cartopy.crs as ccrs

import uxarray as ux

uxgrid = ux.Grid.from_healpix(zoom=3, pixels_only=False)
data = np.sin(np.deg2rad(uxgrid.face_lat.values))
uxda = ux.UxDataArray(data, uxgrid=uxgrid, dims=["n_face"], name="val")

band_edges = np.array([32.0, 52.0])
lat_center = 42.0

z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
z_noncons = uxda.zonal_mean(lat=lat_center)
print("Conservative band mean [32N,52N]:", float(z_cons.values))
print("Non-conservative at 42N:", float(z_noncons.values))

fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
        label=f"lat={lat}°",
    )
lon = uxgrid.face_lon.values
lat = uxgrid.face_lat.values
mask = (lat >= 20) & (lat <= 60) & (lon >= -140) & (lon <= -60)
sc = ax.scatter(
    lon[mask],
    lat[mask],
    c=data[mask],
    s=8,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
ax.set_title("HEALPix face centers with 32–52°N band and center 42°N")
plt.colorbar(sc, ax=ax, orientation="horizontal", pad=0.05, label="val")

Conservative band mean [32N,52N]: 0.6527189003349049
Non-conservative at 42N: 0.6686793861174745


/tmp/ipykernel_4141335/1777304972.py:13: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
/tmp/ipykernel_4141335/1777304972.py:14: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_noncons = uxda.zonal_mean(lat=lat_center)
/tmp/ipykernel_4141335/1777304972.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Conservative band mean [32N,52N]:', float(z_cons.values))
/tmp/ipykernel_4141335/1777304972.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

In [16]:
# Highlight cut faces at band boundaries using face latitude bounds
fb = uxgrid.face_bounds_lat.values  # shape (n_face, 2) [min,max]
cut_lo = (fb[:, 0] < band_edges[0]) & (fb[:, 1] > band_edges[0])
cut_hi = (fb[:, 0] < band_edges[1]) & (fb[:, 1] > band_edges[1])
cut = cut_lo | cut_hi
faces = np.where(cut)[0]

# Draw polygons for a subset around CONUS
fnc = uxgrid.face_node_connectivity.values
node_lon = uxgrid.node_lon.values
node_lat = uxgrid.node_lat.values
fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
for f in faces:
    nodes = fnc[f]
    nodes = nodes[nodes >= 0]
    poly_lon = node_lon[nodes]
    poly_lat = node_lat[nodes]
    ax.plot(
        np.r_[poly_lon, poly_lon[0]],
        np.r_[poly_lat, poly_lat[0]],
        color="crimson",
        linewidth=0.8,
        transform=ccrs.PlateCarree(),
        alpha=0.7,
    )
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
    )
ax.set_title("Faces intersecting 32°N / 52°N band edges (cut region)")

Text(0.5, 1.0, 'Faces intersecting 32°N / 52°N band edges (cut region)')

## HEALPix Band Demonstration (20° band over USA latitudes)

We construct a low-resolution HEALPix grid, define a 20° latitude band spanning roughly the continental USA (32°N–52°N), compare conservative vs non-conservative zonal averaging at the band center (42°N), and visualize the band boundaries.

In [17]:
import cartopy.crs as ccrs

import uxarray as ux

uxgrid = ux.Grid.from_healpix(zoom=3, pixels_only=False)
data = np.sin(np.deg2rad(uxgrid.face_lat.values))
uxda = ux.UxDataArray(data, uxgrid=uxgrid, dims=["n_face"], name="val")

band_edges = np.array([32.0, 52.0])
lat_center = 42.0

z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
z_noncons = uxda.zonal_mean(lat=lat_center)
print("Conservative band mean [32N,52N]:", float(z_cons.values))
print("Non-conservative at 42N:", float(z_noncons.values))

fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
        label=f"lat={lat}°",
    )
lon = uxgrid.face_lon.values
lat = uxgrid.face_lat.values
mask = (lat >= 20) & (lat <= 60) & (lon >= -140) & (lon <= -60)
sc = ax.scatter(
    lon[mask],
    lat[mask],
    c=data[mask],
    s=8,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
ax.set_title("HEALPix face centers with 32–52°N band and center 42°N")
plt.colorbar(sc, ax=ax, orientation="horizontal", pad=0.05, label="val")

Conservative band mean [32N,52N]: 0.6527189003349049
Non-conservative at 42N: 0.6686793861174745


/tmp/ipykernel_4141335/1777304972.py:13: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
/tmp/ipykernel_4141335/1777304972.py:14: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_noncons = uxda.zonal_mean(lat=lat_center)
/tmp/ipykernel_4141335/1777304972.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Conservative band mean [32N,52N]:', float(z_cons.values))
/tmp/ipykernel_4141335/1777304972.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

In [18]:
# Highlight cut faces at band boundaries using face latitude bounds
fb = uxgrid.face_bounds_lat.values  # shape (n_face, 2) [min,max]
cut_lo = (fb[:, 0] < band_edges[0]) & (fb[:, 1] > band_edges[0])
cut_hi = (fb[:, 0] < band_edges[1]) & (fb[:, 1] > band_edges[1])
cut = cut_lo | cut_hi
faces = np.where(cut)[0]

# Draw polygons for a subset around CONUS
fnc = uxgrid.face_node_connectivity.values
node_lon = uxgrid.node_lon.values
node_lat = uxgrid.node_lat.values
fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
for f in faces:
    nodes = fnc[f]
    nodes = nodes[nodes >= 0]
    poly_lon = node_lon[nodes]
    poly_lat = node_lat[nodes]
    ax.plot(
        np.r_[poly_lon, poly_lon[0]],
        np.r_[poly_lat, poly_lat[0]],
        color="crimson",
        linewidth=0.8,
        transform=ccrs.PlateCarree(),
        alpha=0.7,
    )
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
    )
ax.set_title("Faces intersecting 32°N / 52°N band edges (cut region)")

Text(0.5, 1.0, 'Faces intersecting 32°N / 52°N band edges (cut region)')

## HEALPix Band Demonstration (20° band over USA latitudes)

We construct a low-resolution HEALPix grid, define a 20° latitude band spanning roughly the continental USA (32°N–52°N), compare conservative vs non-conservative zonal averaging at the band center (42°N), and visualize the band boundaries.

In [19]:
import cartopy.crs as ccrs

import uxarray as ux

uxgrid = ux.Grid.from_healpix(zoom=3, pixels_only=False)
data = np.sin(np.deg2rad(uxgrid.face_lat.values))
uxda = ux.UxDataArray(data, uxgrid=uxgrid, dims=["n_face"], name="val")

band_edges = np.array([32.0, 52.0])
lat_center = 42.0

z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
z_noncons = uxda.zonal_mean(lat=lat_center)
print("Conservative band mean [32N,52N]:", float(z_cons.values))
print("Non-conservative at 42N:", float(z_noncons.values))

fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
        label=f"lat={lat}°",
    )
lon = uxgrid.face_lon.values
lat = uxgrid.face_lat.values
mask = (lat >= 20) & (lat <= 60) & (lon >= -140) & (lon <= -60)
sc = ax.scatter(
    lon[mask],
    lat[mask],
    c=data[mask],
    s=8,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
ax.set_title("HEALPix face centers with 32–52°N band and center 42°N")
plt.colorbar(sc, ax=ax, orientation="horizontal", pad=0.05, label="val")

Conservative band mean [32N,52N]: 0.6527189003349049
Non-conservative at 42N: 0.6686793861174745


/tmp/ipykernel_4141335/1777304972.py:13: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_cons = uxda.zonal_mean(lat=band_edges, conservative=True)
/tmp/ipykernel_4141335/1777304972.py:14: DeprecationWarning: zonal_mean now returns xarray.DataArray (no grid topology). UxDataArray return is deprecated and will be removed.
  z_noncons = uxda.zonal_mean(lat=lat_center)
/tmp/ipykernel_4141335/1777304972.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Conservative band mean [32N,52N]:', float(z_cons.values))
/tmp/ipykernel_4141335/1777304972.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before per

In [20]:
# Highlight cut faces at band boundaries using face latitude bounds
fb = uxgrid.face_bounds_lat.values  # shape (n_face, 2) [min,max]
cut_lo = (fb[:, 0] < band_edges[0]) & (fb[:, 1] > band_edges[0])
cut_hi = (fb[:, 0] < band_edges[1]) & (fb[:, 1] > band_edges[1])
cut = cut_lo | cut_hi
faces = np.where(cut)[0]

# Draw polygons for a subset around CONUS
fnc = uxgrid.face_node_connectivity.values
node_lon = uxgrid.node_lon.values
node_lat = uxgrid.node_lat.values
fig = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.6)
ax.set_extent([-140, -60, 20, 60], crs=ccrs.PlateCarree())
for f in faces:
    nodes = fnc[f]
    nodes = nodes[nodes >= 0]
    poly_lon = node_lon[nodes]
    poly_lat = node_lat[nodes]
    ax.plot(
        np.r_[poly_lon, poly_lon[0]],
        np.r_[poly_lat, poly_lat[0]],
        color="crimson",
        linewidth=0.8,
        transform=ccrs.PlateCarree(),
        alpha=0.7,
    )
for lat, style, color in [
    (band_edges[0], "-", "red"),
    (band_edges[1], "-", "red"),
    (lat_center, "--", "orange"),
]:
    ax.plot(
        np.linspace(-180, 180, 361),
        np.full(361, lat),
        linestyle=style,
        color=color,
        transform=ccrs.PlateCarree(),
    )
ax.set_title("Faces intersecting 32°N / 52°N band edges (cut region)")

Text(0.5, 1.0, 'Faces intersecting 32°N / 52°N band edges (cut region)')